Импорт

In [1]:
import pandas as pd

Парсинг

In [2]:
column_names_users = ['UserID', 'Gender', 'Age', 'Occupation', 'Zip-code']
df_users = pd.read_csv('data/users.dat', sep='::', header=None, names=column_names_users, engine='python')

df_users

,UserID,Gender,Age,Occupation,Zip-code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455
...,...,...,...,...,...
6035,6036,F,25,15,32603
6036,6037,F,45,1,76006
6037,6038,F,56,1,14706
6038,6039,F,45,0,01060


In [3]:
column_names_rating = ['UserID', 'MovieID', 'Rating', 'Timestamp']
df_rating = pd.read_csv('data/ratings.dat', sep='::', header=None, names=column_names_rating, engine='python')

df_rating

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [4]:
column_names_movies = ['MovieID', 'Title', 'Genres']
df_movies = pd.read_csv('data/movies.dat', sep='::', header=None, names=column_names_movies, engine='python', encoding='utf-16')

df_movies

,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


1. Требуется вычислить средние рейтинги для фильма с ID = № варианта в зависимости
от пола

In [5]:
df_combined = pd.merge(df_rating, df_users[['UserID', 'Gender']], on='UserID')

MOVE_ID = 2959
movie_ratings = df_combined[df_combined['MovieID'] == MOVE_ID]
mean_rating_by_gender = movie_ratings.groupby('Gender')['Rating'].mean()

mean_rating_by_gender


Gender
F    4.020979
M    4.090987
Name: Rating, dtype: float64

2. Отобрать фильмы, получившие не менее 250 оценок, и для них определить средний рейтинг в зависимости от пола;
Результат вывести в виде DataFrame с колонками
['MovieID', 'Title', 'Rating_F', 'Rating_M']


In [6]:
movie_rating_counts = df_combined.groupby(['MovieID']).size().reset_index(name='RateCount')
movie_popular = movie_rating_counts[movie_rating_counts['RateCount'] >= 250]['MovieID']

df_popular = df_combined[df_combined['MovieID'].isin(movie_popular)]
mean_ratings_by_gender = df_popular.groupby(['MovieID', 'Gender'])['Rating'].mean().unstack()

df_gender_rate = pd.merge(mean_ratings_by_gender, df_movies[['MovieID', 'Title']], on='MovieID').rename(columns={'F': 'Rating_F', 'M': 'Rating_M'}).reindex(columns=['MovieID', 'Title', 'Rating_F', 'Rating_M'])

df_gender_rate

,MovieID,Title,Rating_F,Rating_M
0,1,Toy Story (1995),4.187817,4.130552
1,2,Jumanji (1995),3.278409,3.175238
2,3,Grumpier Old Men (1995),3.073529,2.994152
3,5,Father of the Bride Part II (1995),3.212963,2.888298
4,6,Heat (1995),3.682171,3.909988
...,...,...,...,...
1211,3917,Hellraiser (1987),3.022222,3.128205
1212,3927,Fantastic Voyage (1966),3.535714,3.589041
1213,3948,Meet the Parents (2000),3.619048,3.641838
1214,3949,Requiem for a Dream (2000),3.950000,4.174107


3. Найти фильмы, оказавшиеся в топ 5 у зрителей-женщин и худшие 5 у зрителей мужчин. Найти фильмы (топ 5), по которым мужчины и женщины сильнеt всего разошлись в оценках;

In [7]:
df_gender_rate['Rating_Diff'] = abs(df_gender_rate['Rating_F'] - df_gender_rate['Rating_M'])

top_female = df_gender_rate.nlargest(5, 'Rating_F')
worst_male = df_gender_rate.nsmallest(5, 'Rating_M')
top_diff = df_gender_rate.nlargest(5, 'Rating_Diff')

top_diff

,MovieID,Title,Rating_F,Rating_M,Rating_Diff
313,1088,Dirty Dancing (1987),3.790378,2.959596,0.830782
352,1201,"Good, The Bad and The Ugly, The (1966)",3.494949,4.221300,0.726351
1186,3760,"Kentucky Fried Movie, The (1977)",2.878788,3.555147,0.676359
796,2468,Jumpin' Jack Flash (1986),3.254717,2.578358,0.676359
68,231,Dumb & Dumber (1994),2.697987,3.336595,0.638608


4. Определить фильмы (топ 5), вызвавшие в среднем наибольшее разногласие у зрителей независимо от пола;

In [8]:
std_rate_by_movie = df_rating.groupby('MovieID')['Rating'].std()
df_std_rate = pd.merge(std_rate_by_movie, df_movies[['MovieID', 'Title']], on='MovieID')
top_std_rate = df_std_rate.nlargest(5, 'Rating').rename(columns={'Rating': 'std_Rate'}).reindex(columns=['MovieID', 'Title', 'std_Rate'])

top_std_rate

,MovieID,Title,std_Rate
558,572,Foreign Student (1994),2.828427
3557,3800,Criminal Lovers (Les Amants Criminels) (1999),2.309401
133,138,"Neon Bible, The (1995)",2.121320
676,706,Sunset Park (1996),2.121320
828,887,Talk of Angels (1998),2.121320


 5. Определить средний рейтинг каждого жанра для различных возрастов. Установите, фильмы какого жанра в каждой категории возрастов имеют наибольший и наименьший рейтинги.
Если фильм относится к нескольким данным, то их надо разделить и сформировать несколько строчек с этим фильмом, но с различными жанрами, например:

In [24]:
df_movies['Genres'] = df_movies['Genres'].str.split('|').explode('Genres')
df_movies_combined = df_rating.merge(df_users[['UserID', 'Age']], on='UserID').merge(df_movies[['MovieID', 'Genres']], on='MovieID')

genre_age_rate = df_movies_combined.groupby(['Genres', 'Age'])['Rating'].mean().reset_index()

best_genres_by_age = genre_age_rate.loc[genre_age_rate.groupby('Age')['Rating'].idxmax()]
worst_genres_by_age = genre_age_rate.loc[genre_age_rate.groupby('Age')['Rating'].idxmin()]

pivot_table = df_movies_combined.pivot_table(index='Genres', columns='Age', values='Rating')

best_genres_by_age

,Genres,Age,Rating
42,Documentary,1,3.794872
57,Fantasy,18,3.735732
65,Film-Noir,25,3.751695
66,Film-Noir,35,3.788103
67,Film-Noir,45,3.800000
47,Documentary,50,3.896511
48,Documentary,56,3.915625
